### `ENVIRONMENT`

In [1]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 50)

In [2]:
train = pd.read_csv('train_trans.csv')
test = pd.read_csv('test_trans.csv')

### `CATEORICAL FEATURES`

In [3]:
train.head()

,channel_code,main_product_code,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,cluster_code,reservationstatusid_code,resort_id,month_of_arrival,weekday_of_arrival,scheduled_before,week_of_year,day_of_month,week_of_month,amount_spent_per_room_night_scaled
0,3,1,0,46,3,3,3,0.693147,2,7,3,1.098612,5,1,5,2,7,4,3,0.000000,14,5,1,7.706428
1,1,1,0,46,3,3,4,1.791759,2,7,5,1.098612,5,1,5,0,1,4,5,4.369448,15,11,1,6.662563
2,1,1,0,47,1,5,4,1.609438,2,7,1,1.098612,5,1,4,0,9,2,6,1.609438,5,1,0,7.871602
3,1,1,2,46,2,2,3,1.791759,2,7,2,1.609438,5,1,3,0,25,6,3,3.713572,24,11,2,5.344943
4,1,1,0,46,2,2,4,1.791759,2,7,2,1.098612,5,1,3,0,25,12,0,4.644391,51,14,2,7.059346


In [4]:
cat_columns = [0, 1, 3, 4, 5, 6, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22]

### `TRAIN TEST SPLIT`

In [5]:
from sklearn.model_selection import train_test_split

X = train.drop('amount_spent_per_room_night_scaled', axis=1)
y = train.amount_spent_per_room_night_scaled

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=True)

### `PREDICTION MODEL`

In [6]:
from catboost import  CatBoostRegressor

model = CatBoostRegressor(random_seed=0, logging_level='Silent', loss_function='RMSE', eval_metric='RMSE',
                          depth = 8.0, l2_leaf_reg = 5.2643768281007945, learning_rate = 0.15208871780007424,
                          bagging_temperature = 1, od_type = 'Iter', od_wait = 40)

In [7]:
model.fit(x_train, y_train, cat_features = cat_columns, eval_set=(x_test, y_test), use_best_model=True)

In [8]:
model.best_score_

{'learn': {'RMSE': 0.9703087810493881},
 'validation_0': {'RMSE': 0.9886902167518792}}

### `MAKING PREDICTION ON TEST`

In [9]:
z = test.drop('reservation_id', axis=1)
arr = model.predict(z)
arr

array([8.04478107, 7.37666104, 7.58313964, ..., 8.30003283, 8.08213046,
       7.26656321])

In [10]:
test['amount_spent_per_room_night_scaled'] = arr

In [11]:
df_test = test[['reservation_id', 'amount_spent_per_room_night_scaled']]

In [12]:
df_test.to_csv('submission.csv', index=False)